# Triton Inference Server Custom Python Backend Setup

Traditionally for the Triton Inference Server, the Python backend can be used to customize your server with additional requirements (ex: transformers). Generally a conda-pack environment is expected to be packaged in as displayed in this blog: https://aws.amazon.com/blogs/machine-learning/host-ml-models-on-amazon-sagemaker-using-triton-python-backend/. However, this can get heavy having such large artifacts, thus we explore installing these dependencies at the container level itself in this sample.

### Setting
For this example we are working in a SageMaker Classic Notebook Instance (g5.4xlarge), increase the Notebook Volume as needed, the conda-pack and Docker operations will be heavy.

### Credits/Reference
- SageMaker Triton BYOC Example: https://github.com/aws-samples/sagemaker-hosting/tree/main/Inference-Serving-Options/SageMaker-Triton/Triton-BYOC
- SageMaker Python Backend Blog: https://aws.amazon.com/blogs/machine-learning/host-ml-models-on-amazon-sagemaker-using-triton-python-backend/

## Setup

In [16]:
#!pip install nvidia-pyindex
#!pip install tritonclient[http]
#!pip install -U sagemaker numpy transformers 

## Building Docker Image & Setting Up Triton

For the Python backend we will create a conda environment using conda-pack with the necessary dependencies and point towards this in our Dockerfile.

### Conda Env Creation

In [8]:
%%writefile conda_dependencies.sh
conda create -y -n transformers_env python=3.10
source ~/anaconda3/etc/profile.d/conda.sh
source activate transformers_env
export PYTHONNOUSERSITE=True
pip install torch --extra-index-url https://download.pytorch.org/whl/cu116
pip install transformers
pip install conda-pack
conda-pack

Overwriting conda_dependencies.sh


In [2]:
#!bash conda_dependencies.sh # create the conda-pack env

### Build Docker Image

In [6]:
%%writefile Dockerfile
FROM nvcr.io/nvidia/tritonserver:23.12-py3

#Install any additional libraries
RUN echo "Adding conda package to Docker image"
RUN mkdir -p /home/condpackenv/

# Copy conda env
COPY transformers_env.tar.gz /home/condpackenv/transformers_env.tar.gz

Overwriting Dockerfile


#### Docker Build
Run the following command in the Terminal/CLI:

```
docker build -t custom-triton .
```

To check the image has been built, use the following command:

```
docker images
```

## Local Model Inference

For this sample, let's use a BERT transformers model, here's a local Python code inference with it:

In [14]:
# Use a pipeline as a high-level helper
from transformers import pipeline
import numpy as np
model = pipeline("text-classification", model="nlptown/bert-base-multilingual-uncased-sentiment")
output = model("I am super happy")
res = np.array(output)
print(res)

[{'label': '5 stars', 'score': 0.8162010908126831}]


## Triton Artifact Creation

Ensure the following structure is present within the repository:

```
- nlp
    - sentiment (Triton model name)
        - 1
            - model.py (inference logic)
        - config.pbtxt (input/output specs)
```

## Docker Container Startup

To startup the docker container run this command via the terminal or a shell script:

```
docker run --gpus=all --shm-size=4G --rm -p8000:8000 -p8001:8001 -p8002:8002 -v/home/ec2-user/SageMaker/nlp:/model_repository custom-triton:latest tritonserver --model-repository=/model_repository --exit-on-error=false --log-verbose=1
```

Note run this command in the same root directory as the notebook, in this case we are using the SageMaker Notebook Instance home directory path adjust it to your appropriate directory. We also use the "custom-triton" image we built, change this if you named your Docker image otherwise.

## Sample Inference

We can use the Triton Client library or Python requests library for sample inference.

In [18]:
import tritonclient.http as http_client
triton_client = http_client.InferenceServerClient(url="localhost:8000", verbose=True)

In [24]:
import json 
# Create inputs to send to Triton
model_name = "sentiment"
text_inputs = ["I am super happy right now"]

# structure inputs
inputs = []
inputs.append(http_client.InferInput("text", [1], "BYTES"))
input0_real = np.array(text_inputs, dtype=np.object_)
inputs[0].set_data_from_numpy(input0_real)

# structure outputs
outputs = []
outputs.append(http_client.InferRequestedOutput("sent_arr"))

# sample inference
results = triton_client.infer(model_name=model_name, inputs=inputs, outputs=outputs)
sentiment = results.as_numpy('sent_arr')
print(sentiment)

POST /v2/models/sentiment/infer, headers {'Inference-Header-Content-Length': 162}
b'{"inputs":[{"name":"text","shape":[1],"datatype":"BYTES","parameters":{"binary_data_size":30}}],"outputs":[{"name":"sent_arr","parameters":{"binary_data":true}}]}\x1a\x00\x00\x00I am super happy right now'
<HTTPSocketPoolResponse status=200 headers={'content-type': 'application/octet-stream', 'inference-header-content-length': '146', 'content-length': '199'}>
bytearray(b'{"model_name":"sentiment","model_version":"1","outputs":[{"name":"sent_arr","datatype":"BYTES","shape":[1],"parameters":{"binary_data_size":53}}]}')
[b"{'label': '5 stars', 'score': 0.8433139324188232}"]
